In [1]:
from matplotlib import pyplot as plt
import time
import datetime
from utils_laj import *
from data_processing import get_SensorData, my_pca, series_to_supervised, lstm_sampling
from sklearn.model_selection import train_test_split


today = datetime.date.today()

In [2]:
path_checkpoint = './save/save_lstm/lstm_2_layers'
input_path = './input_data'
file = os.path.join(input_path,'sensors.csv')
target_file = os.path.join(input_path,'coke_target.csv')

In [3]:
# Считываем данные из файлов, нормализуем и применяем метод главных комполнентов
X, y, submit_X, mean_y, std_y, pca = get_SensorData(file, target_file, nc=10)


In [4]:
evals = pca.explained_variance_ratio_
evals_cs = evals.cumsum()
print(evals, evals_cs)
# Кумулятивное variance_explained равно 93%

[0.66839234 0.05001844 0.0454892  0.04043455 0.02760575 0.02430107
 0.02172642 0.01984493 0.01803035 0.01471829] [0.66839234 0.71841079 0.76389998 0.80433454 0.83194029 0.85624136
 0.87796778 0.89781272 0.91584307 0.93056136]


In [5]:
# Размерности датасета с известным целевым y и полного датасета
print("Data X: %s, y %s, submit_X %s" % (X.shape, y.shape, submit_X.shape))

Data X: (10400, 11), y (10400, 2), submit_X (13272, 11)


In [6]:
# Формируем сэмплы для LSTM по 250 рядов внахлест с шагом 1
timesteps = 48
Xl, yl = lstm_sampling(X.iloc[:,1:len(list(X))], y=y['target'], timesteps=timesteps)
#yl = y['target'][timesteps:]
print("Data Xl: %s, yl %s" % (Xl.shape, yl.shape))
# формируем данные конкурса по аналогичному принципу
Xs, ys = lstm_sampling(submit_X.iloc[(Xl.shape[0]):,1:len(list(submit_X))], timesteps=timesteps)

# выделим последние 25% в тестовый датасет
x_train, x_test, y_train, y_test = train_test_split(Xl, yl, test_size=0.25, random_state=42, shuffle=False)
print("Data X_train: %s, X_test: %s, y_train %s, y_test %s, Xs %s" % (x_train.shape, x_test.shape, y_train.shape, y_test.shape, Xs.shape))

Samples length 10352
Targets length 10352
Data Xl: (10352, 48, 10), yl (10352, 48)
Samples length 2872
Targets length 0
Data X_train: (7764, 48, 10), X_test: (2588, 48, 10), y_train (7764, 48), y_test (2588, 48), Xs (2872, 48, 10)


In [7]:
    Train = True
    Predict = True
    plot = False

In [8]:
    batch_size = 1024  # Batch size
    if Train == False: batch_size = 1

    sequence_length = timesteps  # Number of steps
    learning_rate = 5*10e-5  # 0.0001
    epochs = 1000
    ann_hidden = 8

    n_channels = x_train.shape[2]

    lstm_size = 24  # Number LSTM units
    num_layers = 2  # 2  # Number of layers

In [9]:
    X = tf.placeholder(tf.float32, [None, sequence_length, n_channels], name='inputs')
    Y = tf.placeholder(tf.float32, [None, sequence_length], name='labels')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    learning_rate_ = tf.placeholder(tf.float32, name='learning_rate')
    is_train = tf.placeholder(dtype=tf.bool, shape=None, name="is_train")

    conv_last_layer = X

    shape = conv_last_layer.get_shape().as_list()
    print('My Conv Shape:',shape)
    CNN_flat = tf.reshape(conv_last_layer, [-1, shape[1] * shape[2]])

    dence_layer_1 = dense_layer(CNN_flat, size=sequence_length * n_channels, activation_fn=tf.nn.relu, batch_norm=False,
                                phase=is_train, drop_out=True, keep_prob=keep_prob,
                                scope="fc_1")
    lstm_input = tf.reshape(dence_layer_1, [-1, sequence_length, n_channels])

    cell = get_RNNCell(['LSTM'] * num_layers, keep_prob=keep_prob, state_size=lstm_size)
    init_state = cell.zero_state(batch_size, tf.float32)
    rnn_output, states = tf.nn.dynamic_rnn(cell, lstm_input, dtype=tf.float32, initial_state=init_state)
    stacked_rnn_output = tf.reshape(rnn_output, [-1, lstm_size])  # change the form into a tensor

    dence_layer_2 = dense_layer(stacked_rnn_output, size=ann_hidden, activation_fn=tf.nn.relu, batch_norm=False,
                                phase=is_train, drop_out=True, keep_prob=keep_prob,
                                scope="fc_2")
    
    #dence_layer_3 = dense_layer(dence_layer_2, size=ann_hidden, activation_fn=tf.nn.relu, batch_norm=False,
    #                            phase=is_train, drop_out=True, keep_prob=keep_prob,
    #                            scope="fc_2_2")

    output = dense_layer(dence_layer_2, size=1, activation_fn=None, batch_norm=False, phase=is_train, drop_out=False,
                         keep_prob=keep_prob,
                         scope="fc_3_output")

    prediction = tf.reshape(output, [-1])
    y_flat = tf.reshape(Y, [-1])

    h = prediction - y_flat

    cost_function = tf.reduce_sum(tf.square(h))
    RMSE = tf.sqrt(tf.reduce_mean(tf.square(h)))
    optimizer = tf.train.AdamOptimizer(learning_rate_).minimize(cost_function)

    saver = tf.train.Saver()
    
    training_generator = batch_generator(x_train, y_train, batch_size, sequence_length, online=True)
    testing_generator = batch_generator(x_test, y_test, batch_size, sequence_length, online=False)

    if Train: model_summary(learning_rate=learning_rate, batch_size=batch_size, lstm_layers=num_layers,
                            lstm_layer_size=lstm_size, fc_layer_size=ann_hidden, sequence_length=sequence_length,
                            n_channels=n_channels, path_checkpoint=path_checkpoint, spacial_note='')
        


My Conv Shape: [None, 48, 10]


In [ ]:
    with tf.Session() as session:
        tf.global_variables_initializer().run()

        if Train == True:
            #saver.restore(session, path_checkpoint)
            #print("Model restored from file: %s" % path_checkpoint)

            cost = []
            iteration = int(x_train.shape[0])
            print("Training set MSE")
            print("No epoches: ", epochs, "No itr: ", iteration)
            __start = time.time()
            for ep in range(epochs):
                for itr in range(iteration):
                    ## training ##
                    batch_x, batch_y = next(training_generator)
                    session.run(optimizer,
                                feed_dict={X: batch_x, Y: batch_y, keep_prob: 0.7, learning_rate_: learning_rate})
                    cost.append(
                        RMSE.eval(feed_dict={X: batch_x, Y: batch_y, keep_prob: 0.7, learning_rate_: learning_rate}))

                x_test_batch, y_test_batch = next(testing_generator)
                mse_train, rmse_train = session.run([cost_function, RMSE],
                                                    feed_dict={X: batch_x, Y: batch_y, keep_prob: 1.0,
                                                               learning_rate_: learning_rate})
                mse_test, rmse_test = session.run([cost_function, RMSE],
                                                  feed_dict={X: x_test_batch, Y: y_test_batch, keep_prob: 1.0,
                                                             learning_rate_: learning_rate})

                time_per_ep = (time.time() - __start)
                time_remaining = ((epochs - ep) * time_per_ep) / 3600
                print("CNNLSTM", "epoch:", ep, "\tTrainig-",
                      "MSE:", mse_train, "RMSE:", rmse_train, "\tTesting-", "MSE", mse_test, "RMSE", rmse_test,
                      "\ttime/epoch:", round(time_per_ep, 2), "\ttime_remaining: ",
                      int(time_remaining), " hr:", round((time_remaining % 1) * 60, 1), " min", "\ttime_stamp: ",
                      datetime.datetime.now().strftime("%Y.%m.%d-%H:%M:%S"))
                __start = time.time()

                if ep % 10 == 0 and ep != 0:
                    save_path = saver.save(session, path_checkpoint)
                    if os.path.exists(path_checkpoint + '.meta'):
                        print("Model saved to file: %s" % path_checkpoint)
                    else:
                        print("NOT SAVED!!!", path_checkpoint)

                if ep % 1000 == 0 and ep != 0: learning_rate = learning_rate / 10

            save_path = saver.save(session, path_checkpoint)
            if os.path.exists(path_checkpoint + '.meta'):
                print("Model saved to file: %s" % path_checkpoint)
            else:
                print("NOT SAVED!!!", path_checkpoint)
            plt.plot(cost)
            plt.show()
        else:
            saver.restore(session, path_checkpoint)
            print("Model restored from file: %s" % path_checkpoint)
            if Predict == True:
                print("Prediction for submit...")
                x_predict = Xs
                y_predict = np.zeros((Xs.shape[0],Xs.shape[1]))

                predict_generator = batch_generator(x_predict, y_predict, batch_size, sequence_length,
                                                       online=True)

                full_prediction = []

                iteration = int(x_predict.shape[0] / (batch_size ))
                print("#of validation points:", x_predict.shape[0], "#datapoints covers from minibatch:",
                      batch_size * sequence_length, "iterations/epoch", iteration)

                for itr in range(iteration):
                    x_validate_batch, y_validate_batch = next(predict_generator)
                    __y_pred, error, __y = session.run([prediction, h, y_flat],
                                                       feed_dict={X: x_validate_batch, Y: y_validate_batch,
                                                                  keep_prob: 1.0})
                    full_prediction.append(__y_pred)
                    print(__y_pred)
                    
                full_prediction = np.array(full_prediction)
                full_prediction = full_prediction.ravel()
            


Training set MSE
No epoches:  1000 No itr:  7764


In [ ]:
Xs.shape

In [ ]:
y_submit = my_gbm.predict(Xs)*std_y+mean_y
sub_file = pd.DataFrame()
sub_file["timestamp"] = submit_X["timestamp"][Xl.shape[0]+time_length:]
sub_file["target"] = y_submit
sub_file.to_csv('submit.csv', index=False)